In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pyqg
import json
import fsspec
import s3fs
import pyqg_parameterization_benchmarks.coarsening_ops as coarsening
from pyqg_parameterization_benchmarks.neural_networks import FCNNParameterization

%matplotlib inline

## Transferring Datasets from LEAP to M²LInES

In [4]:
# Retrieve dataset from LEAP persistent bucket
ds = xr.open_zarr('gs://leap-persistent/pbluc/eddy/high_res', consolidated=True).load()
ds

<xarray.Dataset>
Dimensions:            (run: 5, l: 256, k: 129, lev: 2, time: 87, y: 256,
                        x: 256, lev_mid: 1)
Coordinates:
  * k                  (k) float32 0.0 6.283e-06 ... 0.000798 0.0008042
  * l                  (l) float32 0.0 6.283e-06 ... -1.257e-05 -6.283e-06
  * lev                (lev) int32 1 2
  * lev_mid            (lev_mid) float32 1.5
  * time               (time) float32 0.0 3.6e+06 7.2e+06 ... 3.06e+08 3.096e+08
  * x                  (x) float32 1.953e+03 5.859e+03 ... 9.941e+05 9.98e+05
  * y                  (y) float32 1.953e+03 5.859e+03 ... 9.941e+05 9.98e+05
Dimensions without coordinates: run
Data variables: (12/27)
    APEflux            (run, l, k) float32 -0.0 5.671e-15 ... 5.936e-39
    APEgen             (run) float32 7.518e-11 7.851e-11 ... 8.052e-11 7.41e-11
    APEgenspec         (run, l, k) float32 0.0 -2.749e-15 ... 0.0 -0.0
    Dissspec           (run, l, k) float32 -0.0 -0.0 ... -6.062e-34 -3.081e-35
    EKE                (run, lev) float32 0.002667 7.86e-05 ... 7.948e-05
    EKEdiss            (run) float32 7.277e-11 7.763e-11 ... 7.997e-11 7.359e-11
    ...                 ...
    q                  (run, time, lev, y, x) float32 9.808e-07 ... -9.563e-07
    streamfunction     (run, time, lev, y, x) float32 0.0 0.0 ... -50.38 -49.7
    u                  (run, time, lev, y, x) float32 0.0 0.0 ... -0.009431
    ufull              (run, time, lev, y, x) float32 0.025 0.025 ... -0.009431
    v                  (run, time, lev, y, x) float32 0.0 0.0 ... -0.0002825
    vfull              (run, time, lev, y, x) float32 0.0 0.0 ... -0.0002825
Attributes: (12/24)
    pyqg:L:          1000000.0
    pyqg:M:          65536
    pyqg:W:          1000000.0
    pyqg:beta:       1.5e-11
    pyqg:del2:       0.8
    pyqg:delta:      0.25
    ...              ...
    pyqg:tc:         0
    pyqg:tmax:       311040000.0
    pyqg:twrite:     1000.0
    pyqg_params:     {"nx": 256.0, "dt": 3600.0, "tmax": 311040000.0, "tavest...
    reference:       https://pyqg.readthedocs.io/en/latest/index.html
    title:           pyqg: Python Quasigeostrophic Model

In [5]:
# Write dataset to M²LInES persistent bucket
ds.to_zarr('gs://m2lines-public-persistent/ross22/eddy/high_res', consolidated=True, mode='w')

KeyError: '.zgroup'

In [ ]:
# Read recently written dataset from M²LInES persistent bucket
xr.open_zarr('gs://m2lines-public-persistent/ross22/eddy/high_res', consolidated=True)

## Write Dataset to Persistent Bucket

In [ ]:
# Datasets are temporarily hosted on personal S3 as zarr files
def get_dataset(path, base_url="s3://pyqg-datasets/ross-et-al-2022"):
    fs = s3fs.S3FileSystem(anon=True)
    mapper = fs.get_mapper(f"{base_url}/{path}.zarr")
    return xr.open_zarr(mapper, consolidated=True)

In [15]:
dataset = get_dataset("jet/forcing3") # Last original dataset not yet stored on persistent bucket
dataset

<xarray.Dataset>
Dimensions:            (run: 275, l: 64, k: 33, lev: 2, time: 87, y: 64, x: 64,
                        lev_mid: 1)
Coordinates:
  * k                  (k) float32 0.0 6.283e-06 ... 0.0001948 0.0002011
  * l                  (l) float32 0.0 6.283e-06 ... -1.257e-05 -6.283e-06
  * lev                (lev) int32 1 2
  * lev_mid            (lev_mid) float32 1.5
  * time               (time) float32 0.0 3.6e+06 7.2e+06 ... 3.06e+08 3.096e+08
  * x                  (x) float32 7.812e+03 2.344e+04 ... 9.766e+05 9.922e+05
  * y                  (y) float32 7.812e+03 2.344e+04 ... 9.766e+05 9.922e+05
Dimensions without coordinates: run
Data variables: (12/36)
    APEflux            (run, l, k) float32 dask.array<chunksize=(1, 64, 33), meta=np.ndarray>
    APEgen             (run) float32 dask.array<chunksize=(275,), meta=np.ndarray>
    APEgenspec         (run, l, k) float32 dask.array<chunksize=(1, 64, 33), meta=np.ndarray>
    Dissspec           (run, l, k) float32 dask.array<chunksize=(1, 64, 33), meta=np.ndarray>
    EKE                (run, lev) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    EKEdiss            (run) float32 dask.array<chunksize=(275,), meta=np.ndarray>
    ...                 ...
    uv_subgrid_flux    (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
    v                  (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
    v_subgrid_forcing  (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
    vfull              (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
    vq_subgrid_flux    (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
    vv_subgrid_flux    (run, time, lev, y, x) float32 dask.array<chunksize=(1, 87, 2, 64, 64), meta=np.ndarray>
Attributes: (12/26)
    hires:           256
    lores:           64
    pyqg:L:          1000000.0
    pyqg:M:          4096
    pyqg:W:          1000000.0
    pyqg:beta:       1e-11
    ...              ...
    pyqg:tc:         0
    pyqg:tmax:       311040000.0
    pyqg:twrite:     1000.0
    pyqg_params:     {"nx": 64, "dt": 3600.0, "tmax": 311040000.0, "tavestart...
    reference:       https://pyqg.readthedocs.io/en/latest/index.html
    title:           Subgrid forcing dataset generated by coarsening nx=256 p...

In [ ]:
# Write dataset to LEAP persistent bucket 
dataset.to_zarr('gs://leap-persistent/pbluc/jet/forcing3', consolidated=True, mode='w')

In [ ]:
# Read recently written dataset from LEAP persistent bucket 
xr.open_zarr('gs://leap-persistent/pbluc/jet/forcing3', consolidated=True).load()